In [7]:
import numpy as np
import pandas as pd
import pickle
import ast

from sklearn.model_selection import train_test_split

In [8]:
ratings = pd.read_csv('../the-movies-dataset/ratings_small.csv')
smd = pd.read_csv('../the-movies-dataset/movies_metadata_merge_tmvdb_ver1.csv')
id_map = pd.read_csv('../the-movies-dataset/links_small.csv')[['movieId','tmdbId']]

In [9]:
smd

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,A family wedding reignites the ancient feud be...,1995
3,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Just when George Banks has recovered from his ...,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,40224,False,NaN,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,315011,tt4262980,ja,シン・ゴジラ,...,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,From the mind behind Evangelion comes a hit la...,2016
9095,40503,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,...,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,False,7.6,92.0,"The band stormed Europe in 1963, and, in 1964,...",2016
9096,44821,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0,When Molly Hale's sadness of her father's disa...,2000
9097,44826,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,...,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0,"All your favorite Pokémon characters are back,...",2001


In [10]:
smd = smd.drop(['index'], axis=1)
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,A family wedding reignites the ancient feud be...,1995
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Just when George Banks has recovered from his ...,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,False,NaN,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,From the mind behind Evangelion comes a hit la...,2016
9095,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",...,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,False,7.6,92.0,"The band stormed Europe in 1963, and, in 1964,...",2016
9096,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0,When Molly Hale's sadness of her father's disa...,2000
9097,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0,"All your favorite Pokémon characters are back,...",2001


In [11]:
smd['genres'] = smd['genres'].apply(ast.literal_eval)
smd['genres'] = smd['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [12]:
smd['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

In [13]:
id_map.columns = ['movieId','id']

In [14]:
id_map

,movieId,id
0,1,862.0
1,2,8844.0
2,3,15602.0
3,4,31357.0
4,5,11862.0
...,...,...
9120,162672,402672.0
9121,163056,315011.0
9122,163949,391698.0
9123,164977,137608.0


In [15]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [16]:
ratings = pd.merge(id_map,ratings,left_on='movieId', right_on='movieId')

In [17]:
ratings = ratings[['userId','id','rating','movieId','timestamp']]
ratings

,userId,id,rating,movieId,timestamp
0,7,862.0,3.0,1,851866703
1,9,862.0,4.0,1,938629179
2,13,862.0,5.0,1,1331380058
3,15,862.0,2.0,1,997938310
4,19,862.0,3.0,1,855190091
...,...,...,...,...,...
99999,287,159550.0,5.0,161944,1470167824
100000,73,410612.0,4.5,162376,1474255532
100001,611,392572.0,5.0,162542,1471520667
100002,611,402672.0,3.0,162672,1471523986


In [18]:
ratings = ratings.sort_values(by=['userId'])
ratings

,userId,id,rating,movieId,timestamp
35240,1,152.0,2.5,1371,1260759135
1611,1,9909.0,2.5,31,1260759144
47102,1,847.0,2.0,2193,1260759198
50594,1,9426.0,2.5,2455,1260759113
34595,1,6114.0,3.5,1339,1260759125
...,...,...,...,...,...
66583,671,786.0,2.0,3897,1063503718
65447,671,7443.0,4.0,3751,1065111939
28541,671,1891.0,5.0,1196,1064890635
30829,671,279.0,4.0,1225,1065149143


In [19]:
ratings[np.isnan(ratings['userId'])]

,userId,id,rating,movieId,timestamp


In [20]:
check = ratings[ratings['id'].isnull()].index
ratings = ratings.drop(check)
ratings

,userId,id,rating,movieId,timestamp
35240,1,152.0,2.5,1371,1260759135
1611,1,9909.0,2.5,31,1260759144
47102,1,847.0,2.0,2193,1260759198
50594,1,9426.0,2.5,2455,1260759113
34595,1,6114.0,3.5,1339,1260759125
...,...,...,...,...,...
66583,671,786.0,2.0,3897,1063503718
65447,671,7443.0,4.0,3751,1065111939
28541,671,1891.0,5.0,1196,1064890635
30829,671,279.0,4.0,1225,1065149143


In [21]:
smd[smd['id'] == 58423]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year


In [22]:
tmdb_id = list(ratings.id.unique().astype('int'))
len(tmdb_id)

9053

In [23]:
smd_tmdb_id = list(smd['id'].unique().astype('int'))
len(smd_tmdb_id)

9082

In [24]:
smd = smd.drop(smd[smd.duplicated(['id'])].index)
smd.shape

(9082, 26)

In [25]:
check_id_in_smd = []
for i in smd_tmdb_id:
    if i not in tmdb_id: check_id_in_smd.append(i)
len(check_id_in_smd)        

57

In [26]:
for i in check_id_in_smd:
    check = smd[smd['id'] == i].index
    smd = smd.drop(check)
smd.shape    

(9025, 26)

In [27]:
check_id_in_ratings = []
for i in tmdb_id:
    if i not in smd_tmdb_id: check_id_in_ratings.append(i)
len(check_id_in_ratings)   

28

In [28]:
for i in check_id_in_ratings:
    check = ratings[ratings['id'] == i].index
    ratings = ratings.drop(check)
ratings.shape    

(99810, 5)

In [29]:
ratings

,userId,id,rating,movieId,timestamp
35240,1,152.0,2.5,1371,1260759135
1611,1,9909.0,2.5,31,1260759144
47102,1,847.0,2.0,2193,1260759198
50594,1,9426.0,2.5,2455,1260759113
34595,1,6114.0,3.5,1339,1260759125
...,...,...,...,...,...
66583,671,786.0,2.0,3897,1063503718
65447,671,7443.0,4.0,3751,1065111939
28541,671,1891.0,5.0,1196,1064890635
30829,671,279.0,4.0,1225,1065149143


In [30]:
smd = smd.reset_index(drop=True)

In [31]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,A family wedding reignites the ancient feud be...,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Just when George Banks has recovered from his ...,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9020,False,"{'id': 286023, 'name': 'Sharknado Collection',...",0,"[Comedy, Horror, Science Fiction]",http://www.syfy.com/sharknado4,390989,tt4831420,en,Sharknado 4: The 4th Awakens,The new installment of the Sharknado franchise...,...,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What happens in Vegas, stays in Vegas. Unless ...",Sharknado 4: The 4th Awakens,False,4.3,88.0,The new installment of the Sharknado franchise...,2016
9021,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,85.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,A man must cope with the loss of his wife and ...,2001
9022,False,NaN,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,150.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,"Rustom Pavri, an honourable officer of the Ind...",2016
9023,False,NaN,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,155.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,"Village lad Sarman is drawn to big, bad Mohenj...",2016


In [32]:
def find_movie_index(tmdb_id):
    return smd[smd['id'] == tmdb_id].first_valid_index()

In [ ]:
ratings['movieIndex'] = ratings['id'].apply(lambda x: find_movie_index(x))

In [ ]:
ratings 

In [ ]:
ratings['userId'] = ratings['userId'].apply(lambda x: x -1)
ratings

In [ ]:
train, test = train_test_split(ratings, test_size = 0.2)

In [ ]:
train = train.sort_values(by=['userId'])
test = test.sort_values(by=['userId'])

In [ ]:
train

In [ ]:
ratings['movieIndex'].unique().shape

In [ ]:
test

In [ ]:
train.to_csv(r'../the-movies-dataset/ratings_train.csv', index=False)
test.to_csv(r'../the-movies-dataset/ratings_test.csv', index=False)
ratings.to_csv(r'../the-movies-dataset/ratings_equal_movies_metadata.csv',index=False)
smd.to_csv(r'../the-movies-dataset/movies_metadata_equal_ratings_ver1.csv',index=False)

In [28]:
smd['poster_path']

0       /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1       /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2       /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3       /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4       /e64sOI48hQXyru7naBFyssKFxVd.jpg
5       /zMyfPUelumio3tiDKPffaUpsQTD.jpg
6       /jQh15y5YB7bWz1NtffNZmRw0s9D.jpg
7       /sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg
8       /eoWvKD60lT95Ss1MYNgVExpo5iU.jpg
9       /5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg
10      /lymPNGLZgPHuqM29rKMGV46ANij.jpg
11      /xve4cgfYItnOhtzLYoTwTVy5FGr.jpg
12      /gV5PCAVCPNxlOLFM1bKk50EqLXO.jpg
13      /cICkmCEiXRhvZmbuAlsA5D9B2rK.jpg
14      /odM9973kIv9hcjfHPp6g6BlyTIJ.jpg
15      /xo517ibXBDdYQY81j0WIG7BVcWq.jpg
16      /lA9HTy84Bb6ZwNeyoZKobcMdpMc.jpg
17      /eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg
18      /wRlGnJhEzcxBjvWtvbjhDSU1cIY.jpg
19      /jSozzzVOR2kfXgTUuGnbgG2yRFi.jpg
20      /vWtDUUgQAsVyvRW4mE75LBgVm2e.jpg
21      /80czeJGSoik22fhtUM9WzyjUU4r.jpg
22      /xAx5MP7Dg4y85pyS7atX6eWk4Qd.jpg
23      /1uRKsxOCtgz0xVqs9l4hYtp4dFm.jpg
24      /37qHRJx